In [92]:
%reload_ext autoreload
%autoreload 2

## Leo los datos

In [7]:
from npfd import data

X_train, X_val, y_train, y_val = data.dataset.make_dataset(dataset_name='dmps', test_size=0.2, seed=7)

INFO:root:Converting real raw files to HTK format ...
INFO:root:Generating Master Label File (Train)...
INFO:root:Generating Master Label File (Test)...


In [156]:
mlf = y_val['mlf']
date = '15-02-15'

def mlf_to_dataframe(mlf, date):
    labels = pd.DataFrame()

    with open(mlf, 'rt') as fi:
        tmp_line = fi.readline().splitlines()[0]

        # Encuentro la linea donde empieza la etiqueta
        while not (tmp_line.endswith(date + '.rec"') or tmp_line.endswith(date + '.lab"')):
            tmp_line = fi.readline().splitlines()[0]

        # Leo las etiquetas
        while tmp_line != '.':
            tmp_line = fi.readline().splitlines()[0]
            if tmp_line == '.':
                break
            line = tmp_line.split()
            s = line[0]
            e = line[1]
            l = line[2]
            s = int(int(s) / 60 / 10)
            e = int(int(e) / 60 / 10)
            partial = pd.DataFrame(index=range(s, e), columns=['label'])
            partial.loc[s:e]['label'] = l
            labels = labels.append(partial)

    return labels

In [157]:
mlf_to_dataframe(mlf, date)

,label
0,ne
1,ne
2,ne
3,ne
4,ne
...,...
139,ne
140,ne
141,ne
142,ne


In [173]:
from npfd.models.base import HiddenMarkovModel

vf = 20
mv = 0.02

model = HiddenMarkovModel()
model.initialize(X_train, variance_floor=vf)

model.train(X_train, y_train, minimum_variance=mv, trace=0)

results = model.test(X_val, y_val)

INFO:root:Initializing model...
INFO:root:/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc
INFO:root:Training the model...



/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/notebooks/RPIC
Pruning-Off
Starting Model Update
Model e[1] to be updated with 132 examples
Model ne[2] to be updated with 1116 examples

Pruning-Off
Starting Model Update
Model e[1] to be updated with 132 examples
Model ne[2] to be updated with 1116 examples



INFO:root:Testing model: 3


Pruning-Off
Starting Model Update
Model e[1] to be updated with 132 examples
Model ne[2] to be updated with 1116 examples

/home/gfogwil/Documentos/Facultad/Tesis/programs/htk/HTKTools/HVite -C /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/config -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/3/macros -H /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/models/hmm/3/hmmdefs -p 0.0000000000 -s 1.0000000000 -A -T 0 -S /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A.scp -i /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf -w /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/wdnet /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/dict /home/gfogwil/Documentos/Facultad/Tesis/models/bdb/npfd/models/HTK/misc/monophones 

====================== HTK Results Analysis =======================
  Date: Sat Jul 31 15:55:22 2021
  Ref : >ocumentos/Facultad/Te

{'SENT_Correct': 74.8,
 'SENT_H': 184.0,
 'SENT_S': 62.0,
 'SENT_N': 246.0,
 'WORD_Corr': 75.31,
 'WORD_Acc': 72.19,
 'WORD_H': 241.0,
 'WORD_D': 64.0,
 'WORD_S': 15.0,
 'WORD_I': 10.0,
 'WORD_N': 320.0,
 'mlf': PosixPath('/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/results.mlf')}

In [154]:
def get_metric(mlf_real, mlf_auto, date):
    
    real = data.labels.mlf_to_dataframe(mlf_real, date)
    real.columns = ['label_r']
    real = real.replace({'ne':0, 'e':1})
    
    hmm_prediction = data.labels.mlf_to_dataframe(mlf_auto, date)
    hmm_prediction.columns = ['label_a']
    hmm_prediction = hmm_prediction.replace({'ne':0, 'e':1})

    all_lab = pd.concat([real, hmm_prediction],axis=1)

    m = 0
    for idx, l in all_lab.iterrows():
        if not l['label_r'] and not l['label_a']:
            m += 1
        if not l['label_r'] and l['label_a']:
            m += 0
        if l['label_r'] and not l['label_a']:
            m -= 1
        if l['label_r'] and l['label_a']:
            m += 1
            
    n = all_lab.shape[0]
    
    return m, n

In [155]:
get_metric(y_val['mlf'], results['mlf'], '15-02-15')

0.11805555555555555

In [159]:
import pathlib
scp = pathlib.Path('/home/gfogwil/Documentos/Facultad/Tesis/models/bdb/data/interim/dmps_test_D_A.scp')

In [164]:
for line in scp.open().read().splitlines():
    date = line.split('/')[-1]

13-02-02
18-01-26
20-01-02
20-02-22
15-08-27
19-04-10
18-03-25
16-09-24
18-10-23
16-12-28
19-04-24
16-09-18
19-11-01
17-11-04
17-03-21
19-01-29
20-03-10
19-12-10
17-04-22
16-04-15
16-07-27
17-11-15
18-10-03
18-10-12
18-11-13
18-08-27
15-09-02
18-11-08
16-12-22
15-05-01
18-08-19
18-04-17
19-01-11
19-10-31
15-09-03
16-08-14
15-03-12
19-05-11
19-01-07
17-05-02
20-05-05
17-03-13
20-03-20
20-01-11
17-02-23
15-09-17
16-05-16
16-08-01
16-12-21
14-02-04
17-10-06
19-11-06
16-12-30
20-02-13
15-04-12
19-10-04
16-02-24
18-09-28
18-05-24
19-01-17
20-08-10
19-08-22
18-09-29
17-03-11
19-10-13
16-11-29
16-02-16
19-09-29
17-01-11
15-03-31
15-06-30
15-03-24
20-07-30
16-05-01
15-10-30
19-08-20
20-05-01
15-05-21
18-07-10
20-02-12
18-03-16
20-06-08
19-10-11
18-04-10
20-04-10
19-04-11
18-11-15
15-10-28
18-04-22
18-12-01
16-10-31
17-05-09
17-01-10
18-07-26
19-05-29
20-02-19
17-01-30
18-07-13
20-07-27
16-03-04
17-01-03
16-12-07
18-10-30
20-07-25
18-01-28
18-09-18
19-06-12
16-07-05
17-11-26
19-07-19
17-10-31
1

In [135]:
all_lab

,label_r,label_a
0,0,1
1,0,1
2,0,1
3,0,1
4,0,1
...,...,...
139,0,1
140,0,1
141,0,1
142,0,1


0.11805555555555555

In [12]:
'0 62400 ne'.split()

['0', '62400', 'ne']

144

In [149]:
import pandas as pd

lala = pd.DataFrame(index=[],data=[], columns=['a'])


In [49]:
lala

,a


In [54]:
lala.loc[1:12,'a']=1

In [55]:
lala

,a
